In [1]:
!pip install bert-score transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 726.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [18]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [21]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-fa-base-uncased")

In [22]:
def compute_bertscore(candidates, references, model, tokenizer, lang='fa'):
    candidate_tokens = [tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True) for text in candidates]
    reference_tokens = [tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True) for text in references]

    candidate_embeddings = [model(tokens)[0].squeeze(0) for tokens in candidate_tokens]
    reference_embeddings = [model(tokens)[0].squeeze(0) for tokens in reference_tokens]

    P, R, F1 = bert_score.score(candidates, references, model_type="HooshvareLab/bert-fa-base-uncased", lang='fa', rescale_with_baseline=False)

    return P, R, F1

In [23]:
def get_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    attention_mask = inputs.attention_mask
    return embeddings, attention_mask

In [28]:
def compute_bert_score(candidate_embeddings, reference_embeddings, candidate_mask, reference_mask):
    candidate_embeddings = candidate_embeddings.cpu().numpy()
    reference_embeddings = reference_embeddings.cpu().numpy()

    candidate_mask = candidate_mask.cpu().numpy()
    reference_mask = reference_mask.cpu().numpy()

    similarities = cosine_similarity(candidate_embeddings.reshape(-1, candidate_embeddings.shape[-1]),
                                     reference_embeddings.reshape(-1, reference_embeddings.shape[-1]))

    similarities = similarities.reshape(candidate_embeddings.shape[1], reference_embeddings.shape[1])

    candidate_mask = candidate_mask[0]
    reference_mask = reference_mask[0]

    precision_scores = []
    recall_scores = []

    for i in range(candidate_embeddings.shape[1]):
        if candidate_mask[i] == 0:
            continue
        candidate_sim = similarities[i, :reference_mask.sum()]
        precision = candidate_sim.max()
        precision_scores.append(precision)

    for j in range(reference_embeddings.shape[1]):
        if reference_mask[j] == 0:
            continue
        reference_sim = similarities[:candidate_mask.sum(), j]
        recall = reference_sim.max()
        recall_scores.append(recall)

    precision = np.mean(precision_scores)
    recall = np.mean(recall_scores)
    f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

In [39]:
references = ["من دانشجوی دانشگاه صنعتی شریف هستم."]
candidates = ["من دانشجوی مهندسی کامپیوتر دانشگاه صنعتی اصفهان هستم."]

In [40]:
candidate_embeddings, candidate_mask = get_embeddings(candidates, tokenizer, model)
reference_embeddings, reference_mask = get_embeddings(references, tokenizer, model)

In [ ]:
# P, R, F1 = bert_score.score(
#     cands=candidates,
#     refs=references,
#     model_type=None,
#     num_layers=None,
#     verbose=True,
#     idf=False,
#     device='cuda' if torch.cuda.is_available() else 'cpu',
#     batch_size=64,
#     lang=None,
#     return_hash=False,
#     rescale_with_baseline=False,
#     baseline_path=None,
#     use_fast_tokenizer=True,
#     custom_model=model,
#     custom_tokenizer=tokenizer
# )

In [41]:
precision, recall, f1 = compute_bert_score(candidate_embeddings, reference_embeddings, candidate_mask, reference_mask)

In [42]:
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.8375
Recall: 0.9172
F1 Score: 0.8755
